# Ocean Heat Content - regrid
To ease the computational load, I regridded the yearly averaged temperature and potential density files from the original curvilinear 0.1 degree grid to the a rectangular 0.4 degree grid.

This notebook compared the OHC behaviour of the `ctrl` and `lpd` runs.

In [ ]:
import os
import sys
sys.path.append("..")
import numpy as np
import xarray as xr
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2
%aimport - numpy - scipy - matplotlib.pyplot
matplotlib.rc_file('../rc_file')

In [ ]:
from paths import path_samoc
from xr_regression import xr_quadtrend

In [ ]:
ctrl = xr.open_dataset(f'{path_samoc}/OHC/OHC_integrals_ctrl.nc')
lpd  = xr.open_dataset(f'{path_samoc}/OHC/OHC_integrals_lpd.nc' )
spinup = 50  # how many years to ignore due to spinup effects: data from year 51 of ctrl run

f, ax = plt.subplots(1, 2 , figsize=(8,3), sharey=True)
for i in range(2):  ax[i].axhline(0, c='grey', lw=.5)
    
for i, ocean in enumerate(['Global', 'Atlantic', 'Pacific', 'Southern']):
    key = f'OHC_{ocean}_Ocean'
    c = ['k' ,'C0','C1','C3'][i]
    ax[0].plot(ctrl.time/365, ctrl[key]-ctrl[key][0], c=c, label=ocean)
    ax[1].plot(lpd .time/365, lpd[key]-lpd[key][0], c=c , ls='--')

ax[0].axvspan(0, spinup, alpha=0.3, color='grey')
ax[1].axvspan(lpd.time[0+300-spinup]/365, lpd.time[-1]/365, alpha=0.3, color='grey')

ax[0].legend(frameon=False)

ctrl = ctrl.isel(time=np.arange(50,300))
ctrl = ctrl.assign_coords(time=ctrl.time.values/365)
lpd  = lpd .isel(time=np.arange(0,300-spinup))
lpd  = lpd .assign_coords(time=lpd .time.values/365)

grey shaded areas indicated discarded data

In [ ]:
# adjustment time

In [ ]:
lpd_lat = lpd.TLAT.mean(axis=1)

In [ ]:
f, ax = plt.subplots(1, 2 , figsize=(8,3))
for i, ocean in enumerate(['Global', 'Atlantic', 'Pacific', 'Southern']):
    key = f'OHC_zonal_{ocean}_Ocean'
    c = ['k' ,'C0','C1','C3'][i]
# mean
    ax[0].plot(ctrl.t_lat, ctrl[key][:].mean(dim='time')    , c=c , label=ocean)
    ax[0].plot(lpd_lat   , lpd [key][:].mean(dim='time')/100, c=c , ls='--')

# std of quad. detrended
    ax[1].plot(ctrl.t_lat, (ctrl[key][:]-xr_quadtrend(ctrl[key][:])).std(dim='time'), c=c)
    ax[1].plot(lpd_lat   , (lpd[key] [:]-xr_quadtrend(lpd[key] [:])).std(dim='time')/100, c=c , ls='--')
ax[0].legend(frameon=False)
    

In [ ]:
f, ax = plt.subplots(4, 2 , figsize=(8,12), sharey='row', sharex='col')
cY, cX = np.meshgrid(ctrl.t_lat, ctrl.time)
lY, lX = np.meshgrid(lpd.TLAT.mean(axis=1), lpd.time)
for i, ocean in enumerate(['Global', 'Atlantic', 'Pacific', 'Indian']):
    vex = [3e16, 2e16, 2e16, 1e16][i]
    kwargs = {'cmap':'RdBu', 'vmin':-vex, 'vmax':vex}
    key = f'OHC_zonal_{ocean}_Ocean'
    im = ax[i,0].pcolormesh(cX, cY, ctrl[key] -xr_quadtrend(ctrl[key]), **kwargs)
    ax[i,1].pcolormesh(lX, lY, (lpd[key]-xr_quadtrend(lpd[key]))/100, **kwargs)
    for j in range(2):  ax[i,j].axhline(0, c='grey', lw=.5, ls='--')
    ax[i,0].text(.05, .9, ocean, transform=ax[i,0].transAxes)
    ax[i,0].set_ylabel('latitude')
    ax[i,0].set_yticks(np.arange(-60,100,30))
    if i==0:  ax[i,0].set_ylim((-78,90))
    else:     ax[i,0].set_ylim((-40,90))
ax[0,0].text(.05, .2, 'Southern Ocean', transform=ax[0,0].transAxes)
for i in range(2):
    ax[0,i].axhline(-31.5, c='k')
    ax[-1,i].set_xlabel('time [model years]')
# ax[0].colorbar()

Qualitative difference especially in Pacific around 30N with signals propagating South.
The Atlantic behaviour is qualitatively similar with southwar propagating anomalies in both simulations.
The Southern Ocean shows hints of the SOM in the ctrl run.